In [1]:
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import sklearn
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import  metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV
# importing required libraries

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


In [2]:
df_train = pd.read_csv("train.csv")
df_train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [3]:
df_train.shape

(54808, 14)

In [4]:
df_test = pd.read_csv("test.csv")

df_test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [5]:
drop_train = df_train.drop(["employee_id", "age","recruitment_channel","length_of_service","is_promoted"] , axis = 1)
drop_train.head()

,department,region,education,gender,no_of_trainings,previous_year_rating,KPIs_met >80%,awards_won?,avg_training_score
0,Sales & Marketing,region_7,Master's & above,f,1,5.0,1,0,49
1,Operations,region_22,Bachelor's,m,1,5.0,0,0,60
2,Sales & Marketing,region_19,Bachelor's,m,1,3.0,0,0,50
3,Sales & Marketing,region_23,Bachelor's,m,2,1.0,0,0,50
4,Technology,region_26,Bachelor's,m,1,3.0,0,0,73


In [6]:
drop_test = df_test.drop(["employee_id","recruitment_channel", "age","length_of_service"] , axis = 1)
drop_test.head()

,department,region,education,gender,no_of_trainings,previous_year_rating,KPIs_met >80%,awards_won?,avg_training_score
0,Technology,region_26,Bachelor's,m,1,NaN,1,0,77
1,HR,region_4,Bachelor's,f,1,3.0,0,0,51
2,Sales & Marketing,region_13,Bachelor's,m,1,1.0,0,0,47
3,Procurement,region_2,Bachelor's,f,3,2.0,0,0,65
4,Finance,region_29,Bachelor's,m,1,4.0,0,0,61


In [7]:
drop_train.loc[(drop_train.previous_year_rating.isna()) & (drop_train["KPIs_met >80%"] == 1) , "previous_year_rating"] = 5 
drop_train.loc[(drop_train.previous_year_rating.isna()) & (drop_train["KPIs_met >80%"] == 0) , "previous_year_rating"] = 1

In [8]:
drop_train.isna().sum()

department                 0
region                     0
education               2409
gender                     0
no_of_trainings            0
previous_year_rating       0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [9]:
drop_train.loc[(df_train.age >=39.35) & (df_train.education.isna()), "education"] = "Master's & above"
drop_train.loc[(df_train.age >=23.83)& (df_train.age<39.35) & df_train.education.isna(), "education"] ="Bachelor's"
drop_train.loc[(df_train.age < 23.83) & (df_train.education.isna()), "education" ] ="Below Secondary"


In [10]:
drop_train.isna().sum()

department              0
region                  0
education               0
gender                  0
no_of_trainings         0
previous_year_rating    0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
dtype: int64

In [11]:
dic = {"Bachelor's" : 2 ,"Master's & above" : 3 , "Below Secondary" : 1}
drop_train["education_encoded"] = drop_train.education.map(dic)
drop_train.head()

,department,region,education,gender,no_of_trainings,previous_year_rating,KPIs_met >80%,awards_won?,avg_training_score,education_encoded
0,Sales & Marketing,region_7,Master's & above,f,1,5.0,1,0,49,3
1,Operations,region_22,Bachelor's,m,1,5.0,0,0,60,2
2,Sales & Marketing,region_19,Bachelor's,m,1,3.0,0,0,50,2
3,Sales & Marketing,region_23,Bachelor's,m,2,1.0,0,0,50,2
4,Technology,region_26,Bachelor's,m,1,3.0,0,0,73,2


In [12]:
drop_train.isna().sum()

department              0
region                  0
education               0
gender                  0
no_of_trainings         0
previous_year_rating    0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
education_encoded       0
dtype: int64

In [13]:
drop_test.loc[drop_test.previous_year_rating.isna() & drop_test["KPIs_met >80%"] == 1 , "previous_year_rating"] = 5
drop_test.loc[drop_test.previous_year_rating.isna() & drop_test["KPIs_met >80%"] == 0 , "previous_year_rating"] = 1
drop_test.isna().sum()

department                 0
region                     0
education               1034
gender                     0
no_of_trainings            0
previous_year_rating       0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [14]:

drop_test.loc[(df_test.age >=39.35) & (df_test.education.isna()), "education"] = "Master's & above"
drop_test.loc[(df_test.age >=23.83)& (df_test.age<39.35) & df_test.education.isna(), "education"] ="Bachelor's"
drop_test.loc[(df_test.age < 23.83) & (df_test.education.isna()), "education" ] ="Below Secondary"
drop_test.isna().sum()

department              0
region                  0
education               0
gender                  0
no_of_trainings         0
previous_year_rating    0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
dtype: int64

In [15]:
dic = {"Bachelor's" : 2 ,"Master's & above" : 3 , "Below Secondary" : 1}
drop_test["education_encoded"] = drop_test.education.map(dic)
drop_test.head()

,department,region,education,gender,no_of_trainings,previous_year_rating,KPIs_met >80%,awards_won?,avg_training_score,education_encoded
0,Technology,region_26,Bachelor's,m,1,1.0,1,0,77,2
1,HR,region_4,Bachelor's,f,1,1.0,0,0,51,2
2,Sales & Marketing,region_13,Bachelor's,m,1,1.0,0,0,47,2
3,Procurement,region_2,Bachelor's,f,3,1.0,0,0,65,2
4,Finance,region_29,Bachelor's,m,1,1.0,0,0,61,2


In [16]:
drop_train.dtypes

department               object
region                   object
education                object
gender                   object
no_of_trainings           int64
previous_year_rating    float64
KPIs_met >80%             int64
awards_won?               int64
avg_training_score        int64
education_encoded         int64
dtype: object

In [17]:
drop_train.isna().sum()

department              0
region                  0
education               0
gender                  0
no_of_trainings         0
previous_year_rating    0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
education_encoded       0
dtype: int64

In [18]:
drop_train = drop_train.drop(["education"] , axis = 1)
drop_test = drop_test.drop(["education"] , axis = 1)

In [19]:
drop_train.head()

,department,region,gender,no_of_trainings,previous_year_rating,KPIs_met >80%,awards_won?,avg_training_score,education_encoded
0,Sales & Marketing,region_7,f,1,5.0,1,0,49,3
1,Operations,region_22,m,1,5.0,0,0,60,2
2,Sales & Marketing,region_19,m,1,3.0,0,0,50,2
3,Sales & Marketing,region_23,m,2,1.0,0,0,50,2
4,Technology,region_26,m,1,3.0,0,0,73,2


In [20]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ["department" ,"region"  ,"gender" ]),
    remainder='passthrough')


In [21]:
df_train_old = df_train.is_promoted

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(drop_train,df_train_old , test_size=0.4, random_state=0)


In [23]:
X_train.head()

,department,region,gender,no_of_trainings,previous_year_rating,KPIs_met >80%,awards_won?,avg_training_score,education_encoded
41438,Analytics,region_22,m,1,5.0,0,0,84,2
42120,Sales & Marketing,region_2,m,1,3.0,1,0,46,2
39719,Procurement,region_27,m,3,1.0,1,1,72,2
37142,Analytics,region_2,m,1,5.0,1,0,83,2
40764,Sales & Marketing,region_3,m,1,2.0,0,0,45,3


#Choose all predictors except target & IDcols
predictors = [x for x in drop_train.columns]
print(predictors)
feat_imp = pd.Series(clf.feature_importances_, predictors).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [24]:
X_train.head()

,department,region,gender,no_of_trainings,previous_year_rating,KPIs_met >80%,awards_won?,avg_training_score,education_encoded
41438,Analytics,region_22,m,1,5.0,0,0,84,2
42120,Sales & Marketing,region_2,m,1,3.0,1,0,46,2
39719,Procurement,region_27,m,3,1.0,1,1,72,2
37142,Analytics,region_2,m,1,5.0,1,0,83,2
40764,Sales & Marketing,region_3,m,1,2.0,0,0,45,3


In [25]:
from xgboost import XGBClassifier
import xgboost as xgb

In [26]:
xgb_clf = XGBClassifier()

In [27]:
pipe = make_pipeline(column_trans, xgb_clf)
pipe.fit(drop_train, df_train.is_promoted)


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['department', 'region',
                                                   'gender'])],
                                   verbose=False)),
                ('xgbclassif...
                               interacti

In [28]:
# predict the target on the train dataset
predict_train = pipe.predict(X_train)
print('\nTarget on train data',predict_train) 


# predict the target on the test dataset
predict_test = pipe.predict(X_test)
print('\nTarget on test data',predict_test) 

# Accuracy Score on test dataset
accuracy_test = f1_score(y_test,predict_test)
print('\naccuracy_score on test dataset : ', accuracy_test)


Target on train data [0 0 0 ... 0 0 0]

Target on test data [0 0 0 ... 0 0 0]

accuracy_score on test dataset :  0.546881129854845


In [29]:
prediction = pipe.predict(drop_test)
pred = pd.DataFrame(prediction)
predm = pred.rename({0: "is_promoted"}, axis= 1)
predm.index = df_test.employee_id
predm.to_csv("predictions.csv")

In [30]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='f1',n_jobs=4,iid=False, cv=5)
pipe = make_pipeline(column_trans, gsearch1)
pipe.fit(drop_train, df_train_old)


E:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['department', 'region',
                                                   'gender'])],
                                   verbose=False)),
                ('gridsearch...
                                        

In [51]:
# examine the best model
print(gsearch1.best_score_)
print(gsearch1.best_params_)
print(gsearch1.best_estimator_)

0.5094501096582663
{'max_depth': 7, 'min_child_weight': 1}
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=140, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)


In [55]:
param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='f1',n_jobs=4,iid=False, cv=5)
pipe = make_pipeline(column_trans, gsearch2b)
pipe.fit(drop_train, df_train_old)

E:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['department', 'region',
                                                   'gender'])],
                                   verbose=False)),
                ('gridsearch...
                                        

In [57]:
# examine the best model
print(gsearch2b.best_score_)
print(gsearch2b.best_params_)
print(gsearch2b.best_estimator_)

0.490598551819304
{'min_child_weight': 6}
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=140, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)


In [58]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='f1',n_jobs=4,iid=False, cv=5)
pipe = make_pipeline(column_trans, gsearch3)
pipe.fit(drop_train, df_train_old)

E:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['department', 'region',
                                                   'gender'])],
                                   verbose=False)),
                ('gridsearch...
                                        

In [59]:
# examine the best model
print(gsearch3.best_score_)
print(gsearch3.best_params_)
print(gsearch3.best_estimator_)

0.491092325400854
{'gamma': 0.1}
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=140, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)


In [60]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='f1',n_jobs=4,iid=False, cv=5)
pipe = make_pipeline(column_trans, gsearch4)
pipe.fit(drop_train, df_train_old)

E:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['department', 'region',
                                                   'gender'])],
                                   verbose=False)),
                ('gridsearch...
                                        

In [61]:
# examine the best model
print(gsearch4.best_score_)
print(gsearch4.best_params_)
print(gsearch4.best_estimator_)

0.4972870139068354
{'colsample_bytree': 0.9, 'subsample': 0.8}
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=177, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)


In [66]:
param_test5 = {
 'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
param_grid = param_test5, scoring='f1',n_jobs=4, cv=5)

pipe = make_pipeline(column_trans, gsearch5)
pipe.fit(drop_train, df_train_old)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['department', 'region',
                                                   'gender'])],
                                   verbose=False)),
                ('gridsearch...
                                        

In [67]:
# examine the best model
print(gsearch5.best_score_)
print(gsearch5.best_params_)
print(gsearch5.best_estimator_)

0.49721569076401584
{'colsample_bytree': 0.85, 'subsample': 0.8}
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.85, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=177, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)


In [69]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='f1',n_jobs=4, cv=5)
pipe = make_pipeline(column_trans, gsearch6)
pipe.fit(drop_train, df_train_old)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['department', 'region',
                                                   'gender'])],
                                   verbose=False)),
                ('gridsearch...
                                        

In [71]:
# examine the best model
print(gsearch6.best_score_)
print(gsearch6.best_params_)
print(gsearch6.best_estimator_)

0.4968876008450559
{'reg_alpha': 1}
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=177, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=1,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)


In [72]:
param_test7 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test7, scoring='f1',n_jobs=4,cv=5)
pipe = make_pipeline(column_trans, gsearch7)
pipe.fit(drop_train, df_train_old)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['department', 'region',
                                                   'gender'])],
                                   verbose=False)),
                ('gridsearch...
                                        

In [73]:
# examine the best model
print(gsearch7.best_score_)
print(gsearch7.best_params_)
print(gsearch7.best_estimator_)

0.49446433466670286
{'reg_alpha': 0.005}
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=177, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0.005,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)


In [79]:
# specify "parameter distributions" rather than a "parameter grid"
param_dist =  {'max_depth':range(3,10,2),'min_child_weight':range(1,6,2) ,  'gamma':[i/10.0 for i in range(0,5)] ,
                               'subsample':[i/10.0 for i in range(6,10)],
                                      'colsample_bytree':[i/10.0 for i in range(6,10)] , 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
               , 
}

In [81]:
model = XGBClassifier( learning_rate =0.1, n_estimators=177,subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

In [82]:
rand = RandomizedSearchCV(model, param_dist, cv=10, scoring='f1', n_iter=10, random_state=5, return_train_score=False)
pipe = make_pipeline(column_trans, rand)
pipe.fit(drop_train, df_train_old)
pd.DataFrame(rand.cv_results_)[['mean_test_score', 'std_test_score', 'params']]

,mean_test_score,std_test_score,params
0,0.479139,0.019891,"{'subsample': 0.9, 'reg_alpha': 1, 'min_child_..."
1,0.504126,0.020310,"{'subsample': 0.8, 'reg_alpha': 1, 'min_child_..."
2,0.391619,0.022831,"{'subsample': 0.9, 'reg_alpha': 100, 'min_chil..."
3,0.477308,0.025521,"{'subsample': 0.7, 'reg_alpha': 0.01, 'min_chi..."
4,0.506075,0.018353,"{'subsample': 0.8, 'reg_alpha': 0.01, 'min_chi..."
5,0.509404,0.018716,"{'subsample': 0.7, 'reg_alpha': 1e-05, 'min_ch..."
6,0.504677,0.021054,"{'subsample': 0.6, 'reg_alpha': 1, 'min_child_..."
7,0.470012,0.022946,"{'subsample': 0.6, 'reg_alpha': 1e-05, 'min_ch..."
8,0.504770,0.020898,"{'subsample': 0.8, 'reg_alpha': 1e-05, 'min_ch..."
9,0.354040,0.019194,"{'subsample': 0.9, 'reg_alpha': 100, 'min_chil..."


In [83]:
# examine the best model
print(rand.best_score_)
print(rand.best_params_)
print(rand.best_estimator_)

0.5094037921846691
{'subsample': 0.7, 'reg_alpha': 1e-05, 'min_child_weight': 3, 'max_depth': 9, 'gamma': 0.3, 'colsample_bytree': 0.7}
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.3, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=177, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=1e-05,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.7,
              tree_method=None, validate_parameters=False, verbosity=None)


In [90]:
model = XGBClassifier( learning_rate =0.5, n_estimators=100,subsample=0.7,reg_alpha = 0.005,min_child_weight = 6,max_depth=1 ,
                      gamma = 0.1 , colsample_bytree=0.7,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

In [91]:
pipe = make_pipeline(column_trans, rand)
pipe.fit(drop_train, df_train_old)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['department', 'region',
                                                   'gender'])],
                                   verbose=False)),
                ('randomized...
                                    iid=

In [92]:
# predict the target on the train dataset
predict_train = pipe.predict(X_train)
print('\nTarget on train data',predict_train) 


# predict the target on the test dataset
predict_test = pipe.predict(X_test)
print('\nTarget on test data',predict_test) 

# Accuracy Score on test dataset
accuracy_test = f1_score(y_test,predict_test)
print('\naccuracy_score on test dataset : ', accuracy_test)


Target on train data [0 0 0 ... 0 0 0]

Target on test data [0 0 0 ... 0 0 0]

accuracy_score on test dataset :  0.5433843737730664


In [94]:
prediction = pipe.predict(drop_test)
pred = pd.DataFrame(prediction)
predm = pred.rename({0: "is_promoted"}, axis= 1)
predm.index = df_test.employee_id
predm.to_csv("predictions.csv")